In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from Modules.custom_funx import *
from Modules.plot_utils import *
import lmfit

In [2]:
#--- Loading data ---#
dir_name = 'Data/Stroma/qh03/'
data = load_files(dir_name) 
# print(data)
roi = np.array([(1347,1365),(1774,1800)])
trim_data,cor_spectra, bdata = correct_spectra(data,roi,pol_order = 3)
normSpectra = norm_spectra(trim_data)
# normSpectra


In [3]:
#--- Fit parameters ---#
params = lmfit.Parameters()
algo = 'leastsq'
#               (Name,  Value,   Vary,   Min,     Max,       Expr)
params.add_many(
                # Protein
                ('a1',   0.1,    True,    0,      0.2,     None), #Amplitude
                ('f1',   1740,   True,   1735,    1755,    None), #position
                ('l1',   5,      True,    1,      50,     None), # FWHM

                ('a2',   0.04,    True,     0,      0.2,    None),   #Amplitude
                ('f2',   1725,   True,   1720,     1745,     None),
                ('l2',    5,    True,    1,       50,    None), 
                # Lipid
                ('a3',   0.9,    True,    0,       1,        None),  # Amplitude
                ('f3',   1654,   True,    1640,   1660,     None),
                ('l3',   20,     True,     10,     40,      None),

                ('a4',   0.9,    True,      0,      1,      None), # Amplitude
                ('f4',   1636,   True,   1630,    1650,     None),
                ('l4',   20,     True,     10,     80,      None), 
                
                ####
                ('a5',   0.3,    True,    0.0,     1,       None), # Amplitude
                ('f5',   1547,   True,    1592,    1505,    None),
                ('l5',   40,     True,     0,      100,     None),

                ('a6',   0.7,    True,     0.0,       1,    None), # Amplitude
                ('f6',   1453,   True,    1425,    1470,    None),
                ('l6',   20,     True,     0,      50,      None),   
                
                ('a7',   0.07,    True,     0.0,    1,      None), # Amplitude
                ('f7',   1378,   True,    1350,    1380,    None),
                ('l7',   10,     True,     0,      10,      None),   
                        
               )
print(params.pretty_print())

Name     Value      Min      Max   Stderr     Vary     Expr Brute_Step
a1       0.1        0      0.2     None     True     None     None
a2      0.04        0      0.2     None     True     None     None
a3       0.9        0        1     None     True     None     None
a4       0.9        0        1     None     True     None     None
a5       0.3        0        1     None     True     None     None
a6       0.7        0        1     None     True     None     None
a7      0.07        0        1     None     True     None     None
f1      1740     1735     1755     None     True     None     None
f2      1725     1720     1745     None     True     None     None
f3      1654     1640     1660     None     True     None     None
f4      1636     1630     1650     None     True     None     None
f5      1547     1505     1592     None     True     None     None
f6      1453     1425     1470     None     True     None     None
f7      1378     1350     1380     None     True     None 

In [4]:
df_stats, df_variables, df_residual = run_multifit(normSpectra,params, algo = 'leastsq', message = False)



In [56]:
v = df_variables.loc['y_10']
# v['name']
# yout, peak1,peak2,peak3,peak4,peak5,peak6, peak7 = residual(df_variables.loc['y_10'],normSpectra['x'])
# print(v)
# v.to_dict()
# print(v)
p = v.to_dict()
p

{'name': {'y_10': 'l7'},
 'value': {'y_10': 3.5200396269151177},
 'min': {'y_10': 0},
 'max': {'y_10': 10.0},
 'vary': {'y_10': True},
 'expr': {'y_10': nan},
 'stderr': {'y_10': 9.618855016393724},
 'init_value': {'y_10': 1.5707963267948966}}

In [46]:
params.add_many(p)

TypeError: The DType <class 'numpy._FloatAbstractDType'> could not be promoted by <class 'numpy.dtype[str_]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[str_]'>, <class 'numpy._FloatAbstractDType'>)

In [20]:
x_fit = normSpectra['x']
y_fit = normSpectra['y_10']
result = lmfit.minimize(residual, params, method = algo, args=(x_fit,y_fit))


<class 'lmfit.parameter.Parameters'>
Parameters([('a1', <Parameter 'a1', value=0.046077674991110054 +/- 0.0184, bounds=[0:0.2]>), ('f1', <Parameter 'f1', value=1741.9226470294443 +/- 1.29, bounds=[1735:1755]>), ('l1', <Parameter 'l1', value=7.4067115327232305 +/- 2.4, bounds=[1:50]>), ('a2', <Parameter 'a2', value=0.12837760138251839 +/- 0.0104, bounds=[0:0.2]>), ('f2', <Parameter 'f2', value=1728.0812160257137 +/- 1.9, bounds=[1720:1745]>), ('l2', <Parameter 'l2', value=16.946991678930168 +/- 1.16, bounds=[1:50]>), ('a3', <Parameter 'a3', value=0.6810775355201932 +/- 0.32, bounds=[0:1]>), ('f3', <Parameter 'f3', value=1653.676544541866 +/- 2.58, bounds=[1640:1660]>), ('l3', <Parameter 'l3', value=23.76977280631244 +/- 1.92, bounds=[10:40]>), ('a4', <Parameter 'a4', value=0.4361022014028595 +/- 0.246, bounds=[0:1]>), ('f4', <Parameter 'f4', value=1630.031640062366 +/- 12.7, bounds=[1630:1650]>), ('l4', <Parameter 'l4', value=29.816987152863906 +/- 4.38, bounds=[10:80]>), ('a5', <Parame

In [53]:
print(type(result.params))
result.params


<class 'lmfit.parameter.Parameters'>


name,value,standard error,relative error,initial value,min,max,vary
a1,0.04607767,0.01841908,(39.97%),0.1,0.00000000,0.20000000,True
f1,1741.92265,1.29008627,(0.07%),1740,1735.00000,1755.00000,True
l1,7.40671153,2.39645830,(32.36%),5,1.00000000,50.0000000,True
a2,0.12837760,0.01038774,(8.09%),0.04,0.00000000,0.20000000,True
f2,1728.08122,1.90465913,(0.11%),1725,1720.00000,1745.00000,True
l2,16.9469917,1.15945894,(6.84%),5,1.00000000,50.0000000,True
a3,0.68107754,0.31990776,(46.97%),0.9,0.00000000,1.00000000,True
f3,1653.67654,2.58094103,(0.16%),1654,1640.00000,1660.00000,True
l3,23.7697728,1.91530009,(8.06%),20,10.0000000,40.0000000,True
a4,0.43610220,0.24583444,(56.37%),0.9,0.00000000,1.00000000,True


In [52]:
y_10 = df_variables.loc['y_10']
y_10

,name,value,min,max,vary,expr,stderr,init_value
y_10,a1,0.046078,0,0.2,True,NaN,0.018419,0.000000
y_10,a2,0.128378,0,0.2,True,NaN,0.010388,-0.643501
y_10,a3,0.681078,0,1.0,True,NaN,0.319908,0.927295
y_10,a4,0.436102,0,1.0,True,NaN,0.245834,0.927295
y_10,a5,0.432624,0,1.0,True,NaN,0.004130,-0.411517
y_10,a6,0.514033,0,1.0,True,NaN,0.003161,0.411517
y_10,a7,0.004538,0,1.0,True,NaN,0.007660,-1.035270
y_10,f1,1741.922647,1735,1755.0,True,NaN,1.290086,-0.523599
y_10,f2,1728.081216,1720,1745.0,True,NaN,1.904659,-0.643501
y_10,f3,1653.676545,1640,1660.0,True,NaN,2.580941,0.411517


In [91]:
# v
d= []
for index,row in v.iterrows():
    d.append(row.to_list())
# params = lmfit.Parameters()
# params.add(d[1])
# l1 = list(d.items())
t = tuple(d[1])

('a2',
 0.12837760138251839,
 0,
 0.2,
 True,
 nan,
 0.010387735897525566,
 -0.6435011087932845)

In [74]:
a = (
                # Protein
                ('a1',   0.1,    True,    0,      0.2,     None))

In [85]:
# params = lmfit.Parameters()
# params.add_many(a)
type(j)

list